# Day 1 - Trebuchet

The newly-improved calibration document consists of lines of text; each line originally contained a specific calibration value that the Elves now need to recover. On each line, the calibration value can be found by combining the first digit and the last digit (in that order) to form a single two-digit number.

For example:
```
1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet
```
In this example, the calibration values of these four lines are 12, 38, 15, and 77. Adding these together produces 142.

Consider your entire calibration document. What is the sum of all of the calibration values?

In [ ]:
# Day 1, challenge 1

def calsum(filename):
    calvals = []
    with open(filename, 'r') as file:
        caldoc = file.read()
        first_digit = -1
        second_digit = -1

        #I'm going to just iterate through the whole doc char by char
        # I'm really only looking for two cases, digits and newlines
        for char in caldoc:
            # if it's a digit store it
            if char.isdigit():
                #Just keep updating the second digit every time you see a new one
                second_digit = int(char)
                #Only update the first digit the first time we see one
                if first_digit < 0:
                    first_digit = int(char)
            #if there's a new line or end of file, store the digit for this line and reset
            elif char == '\n' or char == '':
                # First save the calibration number
                calvals.append(first_digit * 10 + second_digit)
                # reset the vars for the digits
                first_digit = -1
                second_digit = -1

        # handle the case if the last line doesn't have a newline or EoF
        if first_digit >= 0 and second_digit >= 0:
            calvals.append(first_digit * 10 + second_digit)
            
    return sum(calvals)

calsum('AoC23-1CalibrationDoc.txt')

# Day 1 - Challenge 2

Your calculation isn't quite right. It looks like some of the digits are actually spelled out with letters: one, two, three, four, five, six, seven, eight, and nine also count as valid "digits".

Equipped with this new information, you now need to find the real first and last digit on each line. For example:
```
two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen
```
In this example, the calibration values are 29, 83, 13, 24, 42, 14, and 76. Adding these together produces 281.

What is the sum of all of the calibration values?


In [ ]:
# Day 1, challenge 2

def calsum2(filename):
    calvals = []
    numbers = ['one','two','three','four','five','six','seven','eight','nine']
    with open(filename, 'r') as file:
        caldoc = file.read()
        first_digit = -1
        second_digit = -1
        doclength = len(caldoc)

        #I'm going to just iterate through the whole doc char by char
        # I'm really only looking for two cases, digits and newlines
        for i,char in enumerate(caldoc):
            if char.isdigit():
                second_digit = int(char)
                if first_digit < 0:
                    first_digit = int(char)
            elif char == '\n' or char == '':
                # First save the calibration number
                calvals.append(first_digit * 10 + second_digit)
                first_digit = -1
                second_digit = -1
            else:
                for n,number in enumerate(numbers):
                    if len(number) + i > doclength:
                        next
                    elif caldoc[i:i+len(number)] == number:
                        second_digit = n + 1
                        if first_digit < 0:
                            first_digit = n + 1
                            

        # handle the case if the last line somehow doesn't have a newline or EoF
        if first_digit >= 0 and second_digit >= 0:
            calvals.append(first_digit * 10 + second_digit)
            
    return sum(calvals)

calsum2('AoC23-1CalibrationDoc.txt')

# Day 2 - Game 1

As you walk, the Elf shows you a small bag and some cubes which are either red, green, or blue. Each time you play this game, he will hide a secret number of cubes of each color in the bag, and your goal is to figure out information about the number of cubes.

To get information, once a bag has been loaded with cubes, the Elf will reach into the bag, grab a handful of random cubes, show them to you, and then put them back in the bag. He'll do this a few times per game.

You play several games and record the information from each game (your puzzle input). Each game is listed with its ID number (like the 11 in Game 11: ...) followed by a semicolon-separated list of subsets of cubes that were revealed from the bag (like 3 red, 5 green, 4 blue).

For example, the record of a few games might look like this:
```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```
In game 1, three sets of cubes are revealed from the bag (and then put back again). The first set is 3 blue cubes and 4 red cubes; the second set is 1 red cube, 2 green cubes, and 6 blue cubes; the third set is only 2 green cubes.

The Elf would first like to know which games would have been possible if the bag contained only 12 red cubes, 13 green cubes, and 14 blue cubes?

In the example above, games 1, 2, and 5 would have been possible if the bag had been loaded with that configuration. However, game 3 would have been impossible because at one point the Elf showed you 20 red cubes at once; similarly, game 4 would also have been impossible because the Elf showed you 15 blue cubes at once. If you add up the IDs of the games that would have been possible, you get 8.

Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. What is the sum of the IDs of those games?

In [ ]:
# Day 2, challenge 1
import re

# Inputs:
# filename - The game file to load
# noisy - Whether to print out a bunch of stuff while running
# target - A dict with keys based on colors and values basec on cube counts that we're testing against
# Returns: sum of game ids that are valid
def cubegame(filename,noisy=True,target={'red':12,'blue':14,'green':13}):
    # this will hold the sum of game ids for those games that were possible
    possible_sum = 0
    # This will tally the maximum number of each color seen in the grabs for a game. It's reset for each game
    game_tally = {'red':0,'blue':0,'green':0}
    
    # re for matching game id
    # Slightly brittle, assumes Game is capitalized
    gamere = re.compile(r'Game\s+(\d+)\:')
    #re for matching the colors in each grab.
    # Slightly brittle, assumes number comes first and nothing is capitalized
    grabre = re.compile(r'(\d+)\s+(red|blue|green)')
    
    with open(filename, 'r') as file:
        # Read line by line
        for line in file:
            # First Grab the Game ID
            game_index = gamere.match(line).group(1)
            
            # Then re match each handful and iterate through the grabs
            # We don't need to separate by the individual grabs (separated by semicolons), because 
            # ... we assume each color appears only once per grab
            for count,color in grabre.findall(line):
                game_tally[color] = max(int(count),int(game_tally[color]))
                
            print(f"Game {game_index}: red {game_tally['red']}, blue {game_tally['blue']}, green {game_tally['green']}") if noisy else None           

            # if the maximum number we saw across the grabs is not greater than the target, we know that target is possible
            if game_tally['red'] <= target['red'] and game_tally['blue'] <= target['blue'] and game_tally['green'] <= target['green']:
                possible_sum += int(game_index)
                print(f"Sum = {possible_sum}") if noisy else None
            game_tally = {'red':'0','blue':'0','green':'0'}
            
    return possible_sum

cubegame('AoC23-2Game.txt',noisy=False)

# Day 2, Game 2

As you continue your walk, the Elf poses a second question: in each game you played, what is the fewest number of cubes of each color that could have been in the bag to make the game possible?

Again consider the example games from earlier:
```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```
    In game 1, the game could have been played with as few as 4 red, 2 green, and 6 blue cubes. If any color had even one fewer cube, the game would have been impossible.
    Game 2 could have been played with a minimum of 1 red, 3 green, and 4 blue cubes.
    Game 3 must have been played with at least 20 red, 13 green, and 6 blue cubes.
    Game 4 required at least 14 red, 3 green, and 15 blue cubes.
    Game 5 needed no fewer than 6 red, 3 green, and 2 blue cubes in the bag.

The power of a set of cubes is equal to the numbers of red, green, and blue cubes multiplied together. The power of the minimum set of cubes in game 1 is 48. In games 2-5 it was 12, 1560, 630, and 36, respectively. Adding up these five powers produces the sum 2286.

For each game, find the minimum set of cubes that must have been present. What is the sum of the power of these sets?

In [ ]:
# Day 2, challenge 2
import re

# Inputs:
# filename - The game file to load
# noisy - Whether to print out a bunch of stuff while running
# target - A dict with keys based on colors (red,green,blue) and values based on cube counts that we're testing against
# Returns: sum of game ids that are valid, sum of power for all games
def cubegame2(filename,noisy=True,target={'red':12,'blue':14,'green':13}):
    # this will hold the sum of game ids for those games that were possible
    possible_sum = 0
    # This will hold the sum of the powers
    power_sum = 0
    # list of colors for iteration purposes.
    # Compared to challenge one version, this would enable us to easily expand to more colors
    colors = ['red','green','blue']
    # This will tally the maximum number of each color seen in the grabs for a game. It's reset for each game
    game_tally = {'red':0,'blue':0,'green':0}
    # This is the difference between the target and the game_tally
    game_diff = {'red':0,'blue':0,'green':0}
    
    #re for matching game id
    gamere = re.compile(r'Game\s+(\d+)\:')
    #re for matching each grab
    # To do: will need to modify this if we add colors
    grabre = re.compile(r'(\d+)\s+(red|blue|green)')
    
    with open(filename, 'r') as file:
        # Read line by line
        for line in file:
            # Reset things. tally to 0 and power to 1 because multiplication
            power = 1
            game_tally = {'red':'0','blue':'0','green':'0'}
            # First Grab the Game ID
            game_index = gamere.match(line).group(1)
            
            # Then re match each handful and iterate through the grabs
            # We don't need to separate by the individual grabs (separated by semicolons), because 
            # ... we assume each color appears only once per grab
            for count,color in grabre.findall(line):
                game_tally[color] = max(int(count),int(game_tally[color]))
                
            print(f"Game {game_index}: red {game_tally['red']}, blue {game_tally['blue']}, green {game_tally['green']}") if noisy else None
            
            # Generalized a bit from above. Instead of specifying red, green blue, iterate through list of colors
            for color in colors:                
                # The game tally is by definition the minimum you'd need for the game so just multiply each color
                power = power * game_tally[color]
                # calculate a diff from the target for testing valid games in challenge 1
                game_diff[color] = target[color] - game_tally[color]

            # Add power to the running sum
            power_sum += power
            print(f"Power = {power}, Power Sum = {power_sum}") if noisy else None
                
            if game_diff['red'] >= 0 and game_diff['blue'] >= 0 and game_diff['green'] >= 0:
                possible_sum += int(game_index)
                print(f"Sum = {possible_sum}") if noisy else None
            
    return possible_sum, power_sum

cubegame2('AoC23-2Game.txt',noisy=False)

# Day 3, Puzzle 1

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [ ]:
# Day 3, challenge 1

def engineparts(filename):
    partsum = 0 
    with open(filename, 'r') as file:

        docmatrix = [line for line in file]
        doclength = len(docmatrix)
        # Just assume all lines are the same length and read the first line here
        for lidx,ln in enumerate(docmatrix):
            ln = ln.rstrip()
            activenumber = -1
            # If I've found a symbol in the active char, the one above or below
            # This will get cleared next time around, unles I'm in a number
            prevsymbol = False
            for cidx,char in enumerate(ln):
                # First check if there's a new symbol here, one below or one above
                chardigit = char.isdigit()
                activesymbol = False
                for y in [-1,0,1]:
                    newidx = lidx+y
                    if newidx < 0 or newidx >= doclength-1:
                        next
                    elif docmatrix[newidx][cidx] != '.' and docmatrix[newidx][cidx].isdigit() == False:
                        activesymbol = True
                if activenumber >=0:
                    #This means I've already started reading in a number
                    #first check if this line is another number
                    if chardigit:
                        activenumber = activenumber * 10 + int(char)
                        prevsymbol = prevsymbol or activesymbol
                        if cidx == len(ln) - 1 and prevsymbol:
                            # We have a countable number here
                            partsum += activenumber
                            activenumber = -1
                            prevsymbol = activesymbol
                    elif activesymbol or prevsymbol:
                        # We have a countable number here
                        partsum += activenumber
                        activenumber = -1
                        prevsymbol = activesymbol
                    else:
                        activenumber = -1
                elif chardigit:
                    activenumber = int(char)
                    prevsymbol = prevsymbol or activesymbol
                else:
                    prevsymbol = activesymbol

        
    return partsum

engineparts('AoC23-3Engine.txt')

# Day 3, Challenge 2

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [ ]:
# Day 3, challenge 2

def combine_symbols(list1, list2):
    if list1 is None:
        return list2
    elif list2 is None:
        return list1
    else:
        return list1 + list2

def engineparts2(filename):
    partsum = 0
    
    # This is going to be a dictionary of all gears
    # The keys are the "id" of the gear, which is a string of the line index - character index
    # The value is a list of all part numbers connected to that gear
    gears = {}
    with open(filename, 'r') as file:

        docmatrix = [line for line in file]
        doclength = len(docmatrix)

        for lidx,ln in enumerate(docmatrix):
            ln = ln.rstrip()

            # This keeps of track of whether we're currently constructing a number
            activenumber = -1
            
            # This keeps track of whather we've seen a symbol before this
            # This either just keeps track of the previous activesymbol (see below) in case a number s starting
            # or if we're in the middle of a part number whether there have been any activesymbols since the beginning the number
            # It's None if there was no symbol in the previous char (or chars if tracking a number)
            # It's an empty list if there have been symbols, but no gears
            # It's a list of gear ids seen
            prevsymbol = None
            for cidx,char in enumerate(ln):
                # First check if there's a new symbol here, one below or one above
                chardigit = char.isdigit()

                # We're first trying to find a symbol in the active char, the one above, or the one below
                # This will get cleared next time around, unles I'm in a number
                # This is None if there are no symbols, an empty list if no gears, or a list of gear ids found
                activesymbol = None
                for y in [-1,0,1]:
                    newidx = lidx+y
                    # first make sure we're not going past the bounds of the data
                    if newidx < 0 or newidx >= doclength-1:
                        next
                    elif docmatrix[newidx][cidx] != '.' and docmatrix[newidx][cidx].isdigit() == False:
                        # We've now found a symbol (not a . or a digit)
                        activesymbol = []
                        if docmatrix[newidx][cidx] == '*':
                            # we have a gear here add the id to the activesymbol
                            gear_index = f'{newidx}-{cidx}'
                            activesymbol.append(gear_index)
                            # also add this key to the global gear dict
                            if gear_index not in gears:
                                gears[gear_index] = []
                if activenumber >=0:
                    #This means I've already started reading in a number
                    #first check if this line is another number
                    if chardigit:
                        #another digit, so keep adding to the active number
                        activenumber = activenumber * 10 + int(char)
                        #since we're mid number, combine symbols in previous and current
                        prevsymbol = combine_symbols(prevsymbol,activesymbol)
                        if cidx == len(ln) - 1 and prevsymbol != None:
                            # We have a countable number here and we've reached the end of the line
                            # Add it to the global part tally
                            partsum += activenumber
                            # Append this number to the gears this number has touched
                            for gidx in prevsymbol:
                                gears[gidx].append(activenumber)
                            #now clear things out
                            activenumber = -1
                            prevsymbol = None

                    # Not another number, so we've reached the end of a number
                    # if we've seen a symbol here or previously store the number
                    elif activesymbol != None or prevsymbol != None:
                        # We have a countable number here
                        partsum += activenumber
                        #even though I'm about to clear out the symbols I need all that this number 
                        # touched so I can append any gears to the global dict
                        prevsymbol = combine_symbols(prevsymbol,activesymbol)
                        for gidx in prevsymbol:
                            gears[gidx].append(activenumber)
                        activenumber = -1
                        # This is to cover the scenario where I've got a single space between two numbers
                        # I don't want keep all of the previous symbols, just the current one
                        # in case a new number starts on the next character
                        prevsymbol = activesymbol
                    else:
                        # not a number and symbols are None
                        # This means a number just ended, but there were no symbols
                        # symbols are already None, so just clear out the active number
                        activenumber = -1
                        
                # we aren't in the middle of a number, so this is the start of a part number
                elif chardigit:
                    activenumber = int(char)
                    prevsymbol = combine_symbols(prevsymbol,activesymbol)
                # Not in the middle of a number or starting a new one
                # Just hold onto symbols at this position in case a number starts next
                else:
                    prevsymbol = activesymbol

        
    #finally calulate gear ratios
    gearsum = 0
    for gidx in gears.keys():
        if len(gears[gidx]) == 2:
            gearsum += gears[gidx][0] * gears[gidx][1]
    
    return partsum, gearsum

engineparts2('AoC23-3Engine.txt')

# Day 4, Challenge 1

The Elf leads you over to the pile of colorful cards. There, you discover dozens of scratchcards, all with their opaque covering already scratched off. Picking one up, it looks like each card has two lists of numbers separated by a vertical bar (|): a list of winning numbers and then a list of numbers you have. You organize the information into a table (your puzzle input).

As far as the Elf has been able to figure out, you have to figure out which of the numbers you have appear in the list of winning numbers. The first match makes the card worth one point and each match after the first doubles the point value of that card.

For example:
```
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
```
In the above example, card 1 has five winning numbers (41, 48, 83, 86, and 17) and eight numbers you have (83, 86, 6, 31, 17, 9, 48, and 53). Of the numbers you have, four of them (48, 83, 17, and 86) are winning numbers! That means card 1 is worth 8 points (1 for the first match, then doubled three times for each of the three matches after the first).

    Card 2 has two winning numbers (32 and 61), so it is worth 2 points.
    Card 3 has two winning numbers (1 and 21), so it is worth 2 points.
    Card 4 has one winning number (84), so it is worth 1 point.
    Card 5 has no winning numbers, so it is worth no points.
    Card 6 has no winning numbers, so it is worth no points.

So, in this example, the Elf's pile of scratchcards is worth 13 points.

Take a seat in the large pile of colorful cards. How many points are they worth in total?

In [ ]:
# Day 4 Challenge 1
import re

def parse_line(line):
    num_winning_numbers = 5
    
    # Pretty simple lines to parse: Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
    # Grab a digit after Card, all digits until the pipe, then all after
    gamere = re.compile(r'Card\s+(\d+)\:\s*((\s|\d)+?)\s*\|\s+((\s|\d)+?)\s*$')
    result = gamere.match(line)
    
    game_no = result.group(1)
    # split out the single strings of numbers into individual numbers
    winning_numbers = re.split(r'\s+',result.group(2))
    my_numbers = re.split(r'\s+',result.group(4))

    return int(game_no), winning_numbers, my_numbers

# Calculate the score based on the rules
# Score is basically 2 to the power of the number of matches cards - 1
def lotto_score(winning_numbers = [], my_numbers = []):
    matches = 0
    for x in my_numbers:
        matches = matches + 1 if x in winning_numbers else matches
        
    
    return 2**(matches - 1) if matches > 0 else 0

def lotto(filename):

    points = 0
    with open(filename, 'r') as file:
        for line in file:
            gn, wn, mn = parse_line(line)
            points += lotto_score(wn,mn)
            
    return points

        
lotto('AoC23-4lotto.txt')

# Day 4, Challenge 2

Just as you're about to report your findings to the Elf, one of you realizes that the rules have actually been printed on the back of every card this whole time.

There's no such thing as "points". Instead, scratchcards only cause you to win more scratchcards equal to the number of winning numbers you have.

Specifically, you win copies of the scratchcards below the winning card equal to the number of matches. So, if card 10 were to have 5 matching numbers, you would win one copy each of cards 11, 12, 13, 14, and 15.

Copies of scratchcards are scored like normal scratchcards and have the same card number as the card they copied. So, if you win a copy of card 10 and it has 5 matching numbers, it would then win a copy of the same cards that the original card 10 won: cards 11, 12, 13, 14, and 15. This process repeats until none of the copies cause you to win any more cards. (Cards will never make you copy a card past the end of the table.)

This time, the above example goes differently:
```
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
```
    Card 1 has four matching numbers, so you win one copy each of the next four cards: cards 2, 3, 4, and 5.
    Your original card 2 has two matching numbers, so you win one copy each of cards 3 and 4.
    Your copy of card 2 also wins one copy each of cards 3 and 4.
    Your four instances of card 3 (one original and three copies) have two matching numbers, so you win four copies each of cards 4 and 5.
    Your eight instances of card 4 (one original and seven copies) have one matching number, so you win eight copies of card 5.
    Your fourteen instances of card 5 (one original and thirteen copies) have no matching numbers and win no more cards.
    Your one instance of card 6 (one original) has no matching numbers and wins no more cards.

Once all of the originals and copies have been processed, you end up with 1 instance of card 1, 2 instances of card 2, 4 instances of card 3, 8 instances of card 4, 14 instances of card 5, and 1 instance of card 6. In total, this example pile of scratchcards causes you to ultimately have 30 scratchcards!

Process all of the original and copied scratchcards until no more scratchcards are won. Including the original set of scratchcards, how many total scratchcards do you end up with?

In [ ]:
# Day 4 Challenge 2
import re

# Simply return the number of matched numbers in the winning numbers
def lotto_matches(winning_numbers = [], my_numbers = []):
    matches = 0
    for x in my_numbers:
        matches = matches + 1 if x in winning_numbers else matches
            
    return matches

def lotto2(filename):

    # Keep a dict of card numbers
    # keys are the card numbers
    # values are the count for that card given the rules
    cards = {}
    # Keep track of the last card number we've seen to ensure we don't go past it
    maxgn = 0
    with open(filename, 'r') as file:
        for line in file:
            gn, wn, mn = parse_line(line)
            matches = lotto_matches(wn,mn)
            maxgn = max(gn,maxgn)
            # add one card for the original card
            cards[gn] = 1 + cards[gn] if gn in cards.keys() else 1
            # add matches number of cards for the matches
            for i in range(gn+1,gn+matches+1):
                cards[i] = cards[gn] + cards[i] if i in cards.keys() else cards[gn] 
    
    # Now clear out any cards we created past the max game
    # At most you can have ten matches, so go five past the max
    for i in range(maxgn+1,maxgn+11):
        if i in cards.keys():
            cards[i] = 0
    return sum(cards.values())

        
lotto2('AoC23-4lotto.txt')

# Day 5, Challenge 1

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:
```
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
```
The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:
```
50 98 2
52 50 48
```
The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:
```
seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
```
With this map, you can look up the soil number required for each initial seed number:

    Seed number 79 corresponds to soil number 81.
    Seed number 14 corresponds to soil number 14.
    Seed number 55 corresponds to soil number 57.
    Seed number 13 corresponds to soil number 13.

The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

    Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
    Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
    Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
    Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.

So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [ ]:
# Day 5 Challenge 1
import re

def parse_seedmap(filename):
    sectionre = re.compile(r'^([\w-]+)\s*(map)?\:\s+(.*)$')
    mapre = re.compile(r'^(\d+)\s+(\d+)\s+(\d+)\s*$')
    # We're first just going to parse the whole file
    with open(filename, 'r') as file:
        activesection = ''
        filesections = {}
        for line in file:
            if activesection == '':
                sectionmatch = sectionre.match(line)
                if sectionmatch:
                    # this is the case where i'm just getting seeds
                    if sectionmatch.group(2) == None:
                        filesections['seeds'] = re.split(r'\s+',sectionmatch.group(3))
                    else:
                        activesection = sectionmatch.group(1)
                        filesections[activesection] = []
                else:
                    next
            else:
                sectionmatch = mapre.match(line)
                if sectionmatch:
                    filesections[activesection].append((int(sectionmatch.group(1)),int(sectionmatch.group(2)),int(sectionmatch.group(3))))
                else:
                    activesection = ''
                    
    return filesections

def source2destination(source,maplist):
    for dmin, smin, l in maplist:
        if source >= smin and source < smin + l:
            offset = source - smin
            return dmin + offset
        
    #if we've gotten here, the source is not in the map
    return source

def seed2location(filename):
    
    # First parse the files
    filesections = parse_seedmap(filename)
    
    # now start looking for the minimum location
    minlocation = 999999999999
    # list of map keys
    # a smarter implementation would parse the key names and figure out the right order
    orderedkeylist = ['seed-to-soil','soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location']    
    for seed in filesections['seeds']:
        source = int(seed)
        for key in orderedkeylist:
            destination = source2destination(source,filesections[key])
            source = destination
        minlocation = min(minlocation,destination)
    
    return minlocation

        
seed2location('AoC23-5Seeds.txt')

Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:

seeds: 79 14 55 13

This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?

In [ ]:
# Day 5, Challenge 2

# Work backwards and provide the source that leads to a destination
def destination2source(destination,maplist):
    for dmin, smin, l in maplist:
        if destination >= dmin and destination < dmin + l:
            offset = destination - dmin
            return smin + offset
        
    #if we've gotten here, the destination is not in the map
    return destination

def seed2location2(filename):
    
    # First parse the files
    filesections = parse_seedmap(filename)
    
    # list of map keys
    # a smarter implementation would parse the key names and figure out the right order
    orderedkeylist = ['seed-to-soil','soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location']    
    
    #I'm going work backwards here. We'll test each location starting at 0 and return when we find one
    # I'll work backwards from location to seed number, then check if that seed is in the given ranges
    for location in range(0,9999999999):
        destination = location
        # loop backward through keys to get back to a seed number
        for key in reversed(orderedkeylist):
            destination = destination2source(destination,filesections[key])
        
        #now I need to loop through my seed ranges to see if I have that seed
        for i in range(0,len(filesections['seeds']),2):
            base = int(filesections['seeds'][i])
            limit = base + int(filesections['seeds'][i+1])
            if destination >= base and destination < limit:
                return location
    
    return -1

        
seed2location2('AoC23-5Seeds.txt')

# Day 6, Challenge 1

You manage to sign up as a competitor in the boat races just in time. The organizer explains that it's not really a traditional race - instead, you will get a fixed amount of time during which your boat has to travel as far as it can, and you win if your boat goes the farthest.

As part of signing up, you get a sheet of paper (your puzzle input) that lists the time allowed for each race and also the best distance ever recorded in that race. To guarantee you win the grand prize, you need to make sure you go farther in each race than the current record holder.

The organizer brings you over to the area where the boat races are held. The boats are much smaller than you expected - they're actually toy boats, each with a big button on top. Holding down the button charges the boat, and releasing the button allows the boat to move. Boats move faster if their button was held longer, but time spent holding the button counts against the total race time. You can only hold the button at the start of the race, and boats don't move until the button is released.

For example:
```
Time:      7  15   30
Distance:  9  40  200
```
This document describes three races:

    The first race lasts 7 milliseconds. The record distance in this race is 9 millimeters.
    The second race lasts 15 milliseconds. The record distance in this race is 40 millimeters.
    The third race lasts 30 milliseconds. The record distance in this race is 200 millimeters.

Your toy boat has a starting speed of zero millimeters per millisecond. For each whole millisecond you spend at the beginning of the race holding down the button, the boat's speed increases by one millimeter per millisecond.

So, because the first race lasts 7 milliseconds, you only have a few options:

    Don't hold the button at all (that is, hold it for 0 milliseconds) at the start of the race. The boat won't move; it will have traveled 0 millimeters by the end of the race.
    Hold the button for 1 millisecond at the start of the race. Then, the boat will travel at a speed of 1 millimeter per millisecond for 6 milliseconds, reaching a total distance traveled of 6 millimeters.
    Hold the button for 2 milliseconds, giving the boat a speed of 2 millimeters per millisecond. It will then get 5 milliseconds to move, reaching a total distance of 10 millimeters.
    Hold the button for 3 milliseconds. After its remaining 4 milliseconds of travel time, the boat will have gone 12 millimeters.
    Hold the button for 4 milliseconds. After its remaining 3 milliseconds of travel time, the boat will have gone 12 millimeters.
    Hold the button for 5 milliseconds, causing the boat to travel a total of 10 millimeters.
    Hold the button for 6 milliseconds, causing the boat to travel a total of 6 millimeters.
    Hold the button for 7 milliseconds. That's the entire duration of the race. You never let go of the button. The boat can't move until you let you of the button. Please make sure you let go of the button so the boat gets to move. 0 millimeters.

Since the current record for this race is 9 millimeters, there are actually 4 different ways you could win: you could hold the button for 2, 3, 4, or 5 milliseconds at the start of the race.

In the second race, you could hold the button for at least 4 milliseconds and at most 11 milliseconds and beat the record, a total of 8 different ways to win.

In the third race, you could hold the button for at least 11 milliseconds and no more than 19 milliseconds and still beat the record, a total of 9 ways you could win.

To see how much margin of error you have, determine the number of ways you can beat the record in each race; in this example, if you multiply these values together, you get 288 (4 * 8 * 9).

Determine the number of ways you could beat the record in each race. What do you get if you multiply these numbers together?



In [ ]:
# Day 6 Challenge 1
import re
import math

def parse_races(filename):
    races = {}
    rowre = re.compile(r'^(\w+)\:\s+(.*)$')
    # We're first just going to parse the whole file
    with open(filename, 'r') as file:
        for line in file:
            rowmatch = rowre.match(line)
            races[rowmatch.group(1)] = re.split(r'\s+',rowmatch.group(2))
            races[rowmatch.group(1)] = list(map(float, races[rowmatch.group(1)]))        
    return races

def raceoptions(time,distance):
    # distance = timeheld*totaltime - timeheld**2
    # th**2-tt*th+d = 0
    # a = 1, b = -tt, c = d
    # th = (-b +- sqrt(b**2 - 4ac)) / 2a
    
    # These will just give me the hold times to equal the current champion
    # This is going to be an upside-down parabola, so there will be a
    # left side which is when distance is increasing with hold time, then a 
    # right side which is when the distance is decreasing with hold time
    # th1 is the hold time to MATCH the record distance on the right side (higher number)
    # th2 is the hold time to MATCH the record distance on the left side (lower number)
    th1 = (time + math.sqrt(time**2 - 4*distance)) / 2
    th2 = (time - math.sqrt(time**2 - 4*distance)) / 2
    
    #This took some trial and error
    # Hold times for some of the records were not whole numbers, but some were
    # The solution worked out to the difference between the seconds to match the record
    # or the cloest value that is short of the record (ceiling for th1, floor for th2)
    # then subtract one
    return math.ceil(th1) - math.floor(th2) - 1

def racesolver(filename):
    answer = 1
    
    # First parse the files
    races = parse_races(filename)
    
    # Just turn into some tuples for ease
    races = zip(races['Time'], races['Distance'])
    
    # Next try the races
    for time, distance in races: 
        answer *= raceoptions(time,distance)
    
    return answer

        
racesolver('AoC23-6Races.txt')

# Day 6, Challenge 2

As the race is about to start, you realize the piece of paper with race times and record distances you got earlier actually just has very bad kerning. There's really only one race - ignore the spaces between the numbers on each line.

So, the example from before:
```
Time:      7  15   30
Distance:  9  40  200
```
...now instead means this:
```
Time:      71530
Distance:  940200

Now, you have to figure out how many ways there are to win this single race. In this example, the race lasts for 71530 milliseconds and the record distance you need to beat is 940200 millimeters. You could hold the button anywhere from 14 to 71516 milliseconds and beat the record, a total of 71503 ways!

How many ways can you beat the record in this one much longer race?



In [ ]:
import re
import math

def parse_races2(filename):
    races = {}
    rowre = re.compile(r'^(\w+)\:\s+(.*)$')
    # We're first just going to parse the whole file
    with open(filename, 'r') as file:
        for line in file:
            rowmatch = rowre.match(line)
            races[rowmatch.group(1)] = int(''.join(re.split(r'\s+',rowmatch.group(2))))
    return races

def racesolver2(filename):
    answer = 1
    
    # First parse the files
    races = parse_races2(filename)
    
    answer = raceoptions(races['Time'],races['Distance'])
    
    return answer

        
racesolver2('AoC23-6Races.txt')

# Day 7 , Challenge 1

Because the journey will take a few days, she offers to teach you the game of Camel Cards. Camel Cards is sort of similar to poker except it's designed to be easier to play while riding a camel.

In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:

    Five of a kind, where all five cards have the same label: AAAAA
    Four of a kind, where four cards have the same label and one card has a different label: AA8AA
    Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
    Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
    Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
    One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
    High card, where all cards' labels are distinct: 23456

Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.

If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand. If these cards are different, the hand with the stronger first card is considered stronger. If the first card in each hand have the same label, however, then move on to considering the second card in each hand. If they differ, the hand with the higher second card wins; otherwise, continue with the third card in each hand, then the fourth, then the fifth.

So, 33332 and 2AAAA are both four of a kind hands, but 33332 is stronger because its first card is stronger. Similarly, 77888 and 77788 are both a full house, but 77888 is stronger because its third card is stronger (and both hands have the same first and second card).

To play Camel Cards, you are given a list of hands and their corresponding bid (your puzzle input). For example:
```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```
This example shows five hands; each hand is followed by its bid amount. Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand. Because there are five hands in this example, the strongest hand will have rank 5 and its bid will be multiplied by 5.

So, the first step is to put the hands in order of strength:

    32T3K is the only one pair and the other hands are all a stronger type, so it gets rank 1.
    KK677 and KTJJT are both two pair. Their first cards both have the same label, but the second card of KK677 is stronger (K vs T), so KTJJT gets rank 2 and KK677 gets rank 3.
    T55J5 and QQQJA are both three of a kind. QQQJA has a stronger first card, so it gets rank 5 and T55J5 gets rank 4.

Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank (765 * 1 + 220 * 2 + 28 * 3 + 684 * 4 + 483 * 5). So the total winnings in this example are 6440.

Find the rank of every hand in your set. What are the total winnings?

In [ ]:
# Day 6 Challenge 1
import re
from collections import defaultdict

def parse_ccfile(filename):
    games = []
    gamere = re.compile(r'^(\w+)\s+(\d+)\s*$')
    # We're first just going to parse the whole file
    with open(filename, 'r') as file:
        for line in file:
            gamematch = gamere.match(line)
            games.append((gamematch.group(1),int(gamematch.group(2))))
    return games

def detecthand(hand):
    # This returns the types numbered and a base10 representation of the hand
    #rather than name the types, we'll just number them
    # 7 = five of a kind
    # 6 = four of a kind
    # 5 = full house
    # 4 = three of a kind
    # 3 = two pair
    # 2 = one pair
    # 1 = high card
    # The base 10 numbering just makes it easier to sort the hands later
    
    carddict = defaultdict(int)
    cardconvert = {'2':0,'3':1,'4':2,'5':3,'6':4,'7':5,'8':6,'9':7,'T':8,'J':9,'Q':10,'K':11,'A':12}
    cardbase10 = 0
    #we'll store each card in a dict where key is the card, value is the count
    for index, card in enumerate(hand):
        carddict[card] += 1
        cardbase10 += cardconvert[card]*(13**(5-index-1))
    
    # This is a list of the card counts sorted from highest to lowest
    # I don't care what the cards are any more, just how many of each
    cardlist = sorted(list(carddict.values()),reverse=True)
    
    # There's probably a better way to do this, but I'm just looking at each type
    # I really only need to look at the highest number I have of any one card
    # except for flush vs. 3 of a kind and 2 pair vs. 1 pair
    cardtype = 1
    if cardlist[0] >= 5:
        cardtype = 7
    elif cardlist[0] == 4:
        cardtype = 6
    elif cardlist[0] == 3 and cardlist[1] == 2:
        cardtype = 5
    elif cardlist[0] == 3 and cardlist[1] <= 1:
        cardtype = 4
    elif cardlist[0] == 2 and cardlist[1] == 2:
        cardtype = 3
    elif cardlist[0] == 2 and cardlist[1] <= 1:
        cardtype = 2
    else:
        cardtype = 1
    
    return cardtype, cardbase10


def camelcards(filename):
    answer = 1
    
    # First parse the files
    hands = parse_ccfile(filename)
    # I'm going to make this a list of tuples
    # first element is a tuple of handtype and the hand in base 10
    # second element is the points
    ranks = []
    # Now let's iterate through the hands
    for hand, wager in hands: 
        ranks.append((detecthand(hand),wager))
  
    # sort all of my hands
    # since tuples are sorted by first element, then second, etc.
    # It will sort by handtype, then the base10 rep of the hand
    # It should never make it to the third element, since that would mean we have two identical hands
    # That seems like an ambigous case
    ranks.sort()
    points = 0
    
    for idx, result in enumerate(ranks):
        # infer rank from the list index and multiply by the score
        points += (idx+1)*result[1]
        
    return points

        
camelcards('AoC23-7Camelcards.txt')

# Day 7, Challenge 2

To make things a little more interesting, the Elf introduces one additional rule. Now, J cards are jokers - wildcards that can act like whatever card would make the hand the strongest type possible.

To balance this, J cards are now the weakest individual cards, weaker even than 2. The other cards stay in the same order: A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J.

J cards can pretend to be whatever card is best for the purpose of determining hand type; for example, QJJQ2 is now considered four of a kind. However, for the purpose of breaking ties between two hands of the same type, J is always treated as J, not the card it's pretending to be: JKKK2 is weaker than QQQQ2 because J is weaker than Q.

Now, the above example goes very differently:

32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483

    32T3K is still the only one pair; it doesn't contain any jokers, so its strength doesn't increase.
    KK677 is now the only two pair, making it the second-weakest hand.
    T55J5, KTJJT, and QQQJA are now all four of a kind! T55J5 gets rank 3, QQQJA gets rank 4, and KTJJT gets rank 5.

With the new joker rule, the total winnings in this example are 5905.

Using the new joker rule, find the rank of every hand in your set. What are the new total winnings?

In [ ]:
# Day 7, Challenge 2

def detecthand2(hand):
    # This returns the types numbered and a base10 representation of the hand
    #rather than name the types, we'll just number them
    # 7 = five of a kind
    # 6 = four of a kind
    # 5 = full house
    # 4 = three of a kind
    # 3 = two pair
    # 2 = one pair
    # 1 = high card
    # this version is changed to handle Jokers as specified in the second set of rules
    
    carddict = defaultdict(int)
    cardconvert = {'J':0,'2':1,'3':2,'4':3,'5':4,'6':5,'7':6,'8':7,'9':8,'T':9,'Q':10,'K':11,'A':12}
    cardbase10 = 0
    jokers = 0
    #we'll store each card in a dict where key is the card, value is the count
    for index, card in enumerate(hand):
        # don't put jokers in the carddict since we'll add them in later
        if card == 'J':
            jokers += 1
        else:
            carddict[card] += 1
        cardbase10 += cardconvert[card]*(13**(5-index-1))
    
    # sort the counts of the non-Jokers
    cardlist = sorted(list(carddict.values()),reverse=True)

    # Just add jokers to the card we have most of
    # However, if we have a hand that is ALL jokers, create a new entry in the card list
    if len(cardlist) > 0:
        cardlist[0] += jokers
    else:
        cardlist.append(jokers)
        
    # There's probably a better way to do this, but I'm just looking at each type
    # I really only need to look at the highest number I have of any one card
    # except for flush vs. 3 of a kind and 2 pair vs. 1 pair
    if cardlist[0] >= 5:
        cardtype = 7
    elif cardlist[0] == 4:
        cardtype = 6
    elif cardlist[0] == 3 and cardlist[1] == 2:
        cardtype = 5
    elif cardlist[0] == 3 and cardlist[1] <= 1:
        cardtype = 4
    elif cardlist[0] == 2 and cardlist[1] == 2:
        cardtype = 3
    elif cardlist[0] == 2 and cardlist[1] <= 1:
        cardtype = 2
    else:
        cardtype = 1
            
    return cardtype, cardbase10


def camelcards2(filename):
    answer = 1
    
    # First parse the files
    hands = parse_ccfile(filename)
    # I'm going to make this a list of tuples
    # first element is a tuple of  hand type, second is a numerical representation of the hand, converted from base 13 to base 10
    # third element is the points
    ranks = []
    # Now let's iterate through the hands
    for hand, wager in hands: 
        ranks.append((detecthand2(hand),wager))
  
    # sort all of my hands
    # since tuples are sorted by first element, then second, etc.
    # It will sort by handtype, then the base10 rep of the hand
    # It should never make it to the third element, since that would mean we have two identical hands
    # That seems like an ambigous case
    ranks.sort()

    points = 0
    for idx, result in enumerate(ranks):
        points += (idx+1)*result[1]
        
    return points

        
camelcards2('AoC23-7Camelcards.txt')